In [41]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,DataCollatorWithPadding
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.amp import autocast
import os
from collections import defaultdict
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [ ]:
# uncomment this to download the training file
# ! curl https://github.com/weiguo-li/Patchscope-Reimplementation/blob/main/Extract_Attricute/output.json -o /kaggle/working/output.json

In [42]:
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())  # Should return True
if torch.cuda.is_available():
    print(torch.cuda.device_count())  # Should return the number of available GPUs
    print(torch.cuda.get_device_name(0))  # Should return the name of the GPU

False


In [43]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_dict = ["Qwen/Qwen2.5-0.5B","Qwen/Qwen2.5-1.5B","Qwen/Qwen2.5-3B"]
model_num = 2
tokenizer = AutoTokenizer.from_pretrained(model_dict[model_num],padding_side = 'left')
model = AutoModelForCausalLM.from_pretrained(model_dict[model_num]).to(device)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm):

#### The data used in this jupyter note nookbook can be found there :https://lre.baulab.info/data/commonsense/

In [44]:
# for n, p in model.named_parameters():
#     print(n)

In [45]:
type(tokenizer)

transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast

In [46]:
# os.listdir("/home/students/wli/Mechanistic_Interpretability/Extract_Attricute/Qwen")

In [47]:
print('hellow world')

hellow world


In [48]:
import os
os.listdir("/kaggle/input/output")

['output.json']

In [49]:
import json
# the first dir is for cluster, the second is for kaggle
wkdir = ['Extract_Attricute/output.json','/kaggle/input/output/output.json']
with open(wkdir[-1], 'r') as f:
    data = json.load(f)

In [50]:
data

{'name': 'fruit outside color',
 'prompt_templates': ['On the outside, {} are'],
 'prompt_templates_zs': ['What color are {} on the outside? They are the color of'],
 'properties': {'relation_type': 'commonsense',
  'domain_name': 'fruit',
  'range_name': 'color',
  'symmetric': False},
 'samples': [{'subject': 'bananas', 'object': 'yellow'},
  {'subject': 'apples', 'object': 'red'},
  {'subject': 'watermelons', 'object': 'green'},
  {'subject': 'kiwis', 'object': 'brown'},
  {'subject': 'dragon fruits', 'object': 'pink'},
  {'subject': 'eggplants', 'object': 'purple'},
  {'subject': 'zucchinis', 'object': 'green'},
  {'subject': 'pineapples', 'object': 'brown'},
  {'subject': 'mangoes', 'object': 'green'},
  {'subject': 'cucumbers', 'object': 'green'},
  {'subject': 'radishes', 'object': 'pink'},
  {'subject': 'passion fruits', 'object': 'purple'},
  {'subject': 'nectarines', 'object': 'red'},
  {'subject': 'plums', 'object': 'purple'},
  {'subject': 'potatos', 'object': 'brown'},
  {

In [51]:
tokenizer(['this is a test','asd'], return_tensors='pt',max_length= 2,padding= True,truncation=True)

{'input_ids': tensor([[   574,    374],
        [151643,  79102]]), 'attention_mask': tensor([[1, 1],
        [0, 1]])}

In [52]:
# to check if each sample can be encoded by the model within the next 20 generated tokens
def check_data(data, model, tokenizer, batch_size=1):
    model.to(device)
    

    max_new_tokens = 21
    
    generated_ids = []

    filtered_data = []

    prompt_tmp = 'One the outside, the color of '
    prompt_baselines = []
    for i in range(len(data['samples'])):
            prompt_baseline = prompt_tmp + data['samples'][i]['subject']
            prompt_baselines.append(prompt_baseline)

    num_batches = len(prompt_baselines) / batch_size



    if len(prompt_baselines) % batch_size == 0:
        pass
    else:
        num_batches = int(num_batches) + 1


    for i in range(num_batches):
        start_idx = i * num_batches
        end_idx = min(start_idx + batch_size, len(prompt_baselines))

        batched_prompts = prompt_baselines[start_idx:end_idx]

        model_inputs = tokenizer(batched_prompts, return_tensors='pt', padding= True).to(device)
        generated_id_batch = model.generate(**model_inputs,
                max_new_tokens = max_new_tokens
                )
        docoded_outputs= tokenizer.batch_decode(generated_id_batch,skip_special_tokens=True)


        for j in range(len(docoded_outputs)):
             if data['samples'][start_idx + j]['object']  in docoded_outputs[j]:
                  filtered_data.append(data['samples'][start_idx+j])
                  
        
    return filtered_data
             
             



from the result above, we can see that Qwen 2.5-3B is better than GPT-J - 6B

In [53]:
filtered_data = check_data(data,model,tokenizer,16)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [54]:
print(filtered_data)
print(f" the lenght of filtered_data is {len(filtered_data)}")

[{'subject': 'bananas', 'object': 'yellow'}, {'subject': 'apples', 'object': 'red'}, {'subject': 'eggplants', 'object': 'purple'}, {'subject': 'zucchinis', 'object': 'green'}, {'subject': 'cucumbers', 'object': 'green'}, {'subject': 'grapefruits', 'object': 'orange'}, {'subject': 'eggplants', 'object': 'purple'}, {'subject': 'zucchinis', 'object': 'green'}, {'subject': 'cucumbers', 'object': 'green'}, {'subject': 'grapefruits', 'object': 'orange'}, {'subject': 'limes', 'object': 'green'}, {'subject': 'oranges', 'object': 'orange'}]
 the lenght of filtered_data is 12


In [55]:
# for each filtered_data

# sample 

import dataset

In [56]:
from datasets import load_dataset

ds = load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1",split='train',streaming=True)
ds.shuffle(seed = 50)

IterableDataset({
    features: ['text'],
    num_shards: 2
})

In [57]:
ds_it = iter(ds)

In [58]:
next(ds_it)

{'text': ''}

In [59]:
list(ds.take(5))

[{'text': ''},
 {'text': ' = Valkyria Chronicles III = \n'},
 {'text': ''},
 {'text': ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n'},
 {'text': " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent mult

In [60]:
import random
def sample_utterance(sigma, dataset, num_samples = 5):
    
    matching_lines = [line['text'].lower() for line in dataset if sigma in line['text'].lower()]
    if len(matching_lines) < num_samples:
        print("not enough utterance")
        sampled_utterance = matching_lines
    else:
        sampled_utterance = random.sample(matching_lines,num_samples)

    
    return sampled_utterance

def truncat_to_window(utterance, sigma,tokenizer, window_size = 20,last_target_flag : bool = False):

    """
    utterance : List[Dict['text': str]]  with_size being 5 by default
    
    in order to patch the hidden representation of the last subject's token to the target model, 
    I choose to set subject to be the last few tokens of sampled_utterance.
    Cause tokenizer use the Byte encoding wordpiece

    return : list[(token_id, offset, sigma_len)]

    """
    
    sigma_tokens = tokenizer.tokenize(sigma)

    sigma_len = len(sigma_tokens)

    prefix_len = 3

    input_ids =[]

    for line in utterance:
        tokens = tokenizer.tokenize(line)

        # find the start index of sigma_tokens
        sigma_start = 0

        for i in range(len(tokens)-sigma_len + 1):

            if tokenizer.convert_tokens_to_string(tokens[i:i + sigma_len]).lower() == tokenizer.convert_tokens_to_string(sigma_tokens).lower():
                sigma_start = i
                break
        
        # print(sigma_start)
        # I tend to keep some prefix before suject as possible so that 
        max_start = max(0,  sigma_start - prefix_len)
        min_start = max(0, sigma_start - window_size + sigma_len)


        start_idx = random.randint(min_start, max_start)
        if last_target_flag:
            end_idx = sigma_start + sigma_len
     
        else:
            end_idx = min(start_idx + window_size, len(line))

        # print(start_idx, end_idx) # for debug reasoning

        truncated_tokens = tokens[start_idx:end_idx]  # [ids]

        input_ids.append((truncated_tokens,(sigma_start - start_idx, sigma_len)))

    return input_ids

In [61]:
# a = sample_utterance(' bananas', ds)

In [62]:
# a

In [63]:
# truncat_to_window(a,' bananas',tokenizer)

In [64]:
# for line in truncat_to_window(a,' bananas',tokenizer):
#     print(tokenizer.convert_tokens_to_string(line[0]))
#     print(line[0])
#     print(line[1])
#     print(tokenizer.convert_tokens_to_ids(line[0]))
#     print(" $$$$$$$$")

' banana' is not in 'banana'
space is very important


In [65]:
filtered_data

[{'subject': 'bananas', 'object': 'yellow'},
 {'subject': 'apples', 'object': 'red'},
 {'subject': 'eggplants', 'object': 'purple'},
 {'subject': 'zucchinis', 'object': 'green'},
 {'subject': 'cucumbers', 'object': 'green'},
 {'subject': 'grapefruits', 'object': 'orange'},
 {'subject': 'eggplants', 'object': 'purple'},
 {'subject': 'zucchinis', 'object': 'green'},
 {'subject': 'cucumbers', 'object': 'green'},
 {'subject': 'grapefruits', 'object': 'orange'},
 {'subject': 'limes', 'object': 'green'},
 {'subject': 'oranges', 'object': 'orange'}]

In [66]:
from torch.utils.data import DataLoader
from datasets import Dataset

In [67]:
config = model.config
num_layers = config.num_hidden_layers

target_prompt = "On the outside, the color of x"
inputs = []

In [68]:
! pip install tqdm

In [69]:
data_to_eval = []
from tqdm import tqdm
for item in tqdm(filtered_data, total = len(filtered_data)):
    inputs = []
    subj = ' ' +  item['subject']  # becasue of the tokenizer's property, ' ' should be added. To keep consistency 
    obj = item['object']

    sampled_subj = sample_utterance(subj,load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1",split='train',streaming=True))

    processed_sampled_utterance = truncat_to_window(sampled_subj, subj,tokenizer,last_target_flag=True)


    for tokens, sigma_pos in processed_sampled_utterance:
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        
        inputs.append({'input_ids':input_ids,'obj':obj})
        # inputs['input_ids']append(input_ids)
        # inputs['object'].append(obj)
        # inputs['target_pos'].append(sigma_pos)

    data_to_eval.append(Dataset.from_list(inputs))
    break
# eval_data = Dataset.from_list(inputs)

  0%|          | 0/12 [01:51<?, ?it/s]


In [70]:
len(data_to_eval)

1

In [71]:
# file out those data that has less than five utterance
data_to_eval_filtered = [data for data in data_to_eval if len(data) == 5]

In [72]:
data_to_eval_filtered[0]['obj'][0]

'yellow'

In [73]:
# eval_data.save_to_disk('./')

In [74]:
# eval_data = Dataset.load_from_disk('./')

In [75]:
tokenizer(' bananas')

{'input_ids': [67342], 'attention_mask': [1]}

In [76]:
# collator = DataCollatorWithPadding(tokenizer)
# dataloader = DataLoader(
#     eval_data.remove_columns(['obj']),
#     4,
#     collate_fn=collator
# )

In [77]:
# this cell only do "one to five"
def cal_acc(decoded_output:list,obj:str, flag_record : defaultdict, layer_idx):
    
    for idx, output in enumerate(decoded_output):
        if obj in output.lower():
            assert idx < 5 and layer_idx == 0,   f"idx is {idx}"
            flag_record[layer_idx][idx] =True
            
def patchscope(dataloader,obj):
    batch_size = 5
    obj = obj       
    flag_record  = defaultdict(list)
    # first forward pass on source prompts

    with torch.no_grad():
        for batch in dataloader:
            batch.to(device)
            
            # print(batch)
            print(f" the batch size is {len(batch['input_ids'])}")
            
            # first forward pass on source prompts
            print(batch)
            source_output = model(input_ids = batch['input_ids'],attention_mask = batch['attention_mask'], output_hidden_states = True)
            # print(source_output)
            source_hidden_state = source_output.hidden_states[1:]  # exclude embedding layer
            
            #second forward pass on target prompts
            for layer_idx, layer in enumerate(source_hidden_state):
                flag_record[layer_idx] = [False] * 5
                
                # print(flag_record)
                # print(f"the size of source hidden states is {layer.shape}")
    
                source_layer_hidden_states = layer[:,-1,:]   #(batch_size, sequence, hidden_dimension)
    
                for layer_target in range(num_layers):
    
                    # print(f"the size of patched hidden state is {layer_hidden_states.shape}")
    
                    first_forwardpass_flag = True  # to control only in prefill stage can we pathch the hidden states 
                                                    # otherwise the forward hook will be called many times.
                    
                    def forward_hook(module, input, output):
                        nonlocal first_forwardpass_flag
                        # print(first_forwardpass_flag)
                        if first_forwardpass_flag:
    
                            print(f"the size of output is ************************************************{output[0].shape}")
                            # print(f"the size of layer hidden states {source_layer_hidden_states.shape}")
                            output[0][: , -1 , :] = source_layer_hidden_states
    
                            first_forwardpass_flag = False
                            
    
                    hook_handle = model.model.layers[layer_target].register_forward_hook(forward_hook)
                    generated_ids = model.generate(**tokenizer([target_prompt for i in range(len(batch['input_ids']))],padding = True,return_tensors='pt').to(device),max_new_tokens = 21)
                    hook_handle.remove()
    
                    decoded_output = tokenizer.batch_decode(generated_ids,skip_special_tokens=True)
                    
                    print(decoded_output)
    
                    cal_acc(decoded_output,obj,flag_record,layer_idx)
                    
                    # break
                break
    
            break

    return flag_record

In [78]:
collator = DataCollatorWithPadding(tokenizer)
for data in tqdm(data_to_eval_filtered,total = len(data_to_eval_filtered)):
    dataloader = DataLoader(
    data.remove_columns(['obj']),
    5,
    collate_fn=collator)
    
    patchscope(dataloader,data['obj'][0])
    break
    

  0%|          | 0/1 [00:00<?, ?it/s]

 the batch size is 5
{'input_ids': tensor([[ 29949,    320,   5310,  87822,    323,  23875,    610,   3530,    873,
           1154,  34167,   1154,  58882,  43627,   1154,  67342],
        [151643, 151643, 151643,    320,  23880,    525,   8789,    537,  66066,
            659,    873,  16626,  34561,  25322,   2924,  67342],
        [151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,    315,
          10971,  41891,    220,     20,    686,    728,  67342],
        [151643,   7040,  10785,   2471,    916,   3460,  26552,    659,    279,
          49637,    315,   5749,   1280,  13041,   1154,  67342],
        [151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643,    220,     20,    481,  67342]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1, 1, 1, 1, 

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


the size of output is ************************************************torch.Size([5, 8, 2048])


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

['On the outside, the color of x 10000000000000000000', 'On the outside, the color of x (x is a positive integer) is red. If x is odd, then the color of x+', 'On the outside, the color of x 10000000000000000000', 'On the outside, the color of x 10000000000000000000', 'On the outside, the color of x (x is a positive integer) is red. If x is odd, then the color of x+']
the size of output is ************************************************torch.Size([5, 8, 2048])


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


['On the outside, the color of xSPATH is a deep, rich, and dark brown. On the inside, it is a light, creamy', 'On the outside, the color of xSPATH is a deep, rich, and dark brown. On the inside, it is a light, creamy', 'On the outside, the color of xSPATH is a deep, rich, and dark brown. On the inside, it is a light, creamy', 'On the outside, the color of xSPATH is a deep, rich, and dark brown. On the inside, it is a light, creamy', 'On the outside, the color of xSPATH is a deep, rich, and dark brown. On the inside, it is a light, creamy']
the size of output is ************************************************torch.Size([5, 8, 2048])


  0%|          | 0/1 [30:23<?, ?it/s]

['On the outside, the color of xnewValue is the same as the color of xOldValue. On the inside, the color of xnewValue', 'On the outside, the color of xnewValue is the same as the color of xOldValue. On the inside, the color of xnewValue', 'On the outside, the color of xnewValue is the same as the color of xOldValue. On the inside, the color of xnewValue', 'On the outside, the color of xnewValue is the same as the color of xOldValue. On the inside, the color of xnewValue', 'On the outside, the color of xalore is a deep, rich, dark brown. On the inside, it is a light, creamy white']


In [82]:
target_prompt

'On the outside, the color of x'

In [81]:
batch_size = 2
tokenizer([target_prompt for i in range(batch_size)],padding = True)

{'input_ids': [[1925, 279, 4889, 11, 279, 1894, 315, 856], [1925, 279, 4889, 11, 279, 1894, 315, 856]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
# Example batch of input IDs
input_ids_list = [
    [101, 7592, 102],          # "Hello"
    [101, 7632, 2045, 102],    # "Hi there"
    [101, 1045, 1005, 2310, 102]  # "I’ve"
]

pp = [{'input_ids' : ids_list,'abc':(0,1,22)}  for ids_list in input_ids_list]

In [ ]:
pp[0 : 2]

In [ ]:
cc = tokenizer.tokenize(' apples')
print(cc)

In [ ]:
dd = tokenizer.tokenize(' apples')
print(dd)

The biggest issue here is that 'Bananas' and 'bananas' are tookenized in a different way
And ' bananas' and 'bananas' are tokenized differently 

So I change my code a little bit so that we only sample those lines which has the strict match with subject

In [ ]:
ee = tokenizer.convert_tokens_to_string(cc).lower()
print(ee)

In [ ]:
ff = tokenizer.convert_tokens_to_string(dd).lower()
print(ff)

In [ ]:
ee == ff